# Overview


Data Source:

1. Energy Data: https://www.kaggle.com/datasets/lislejoem/us_energy_census_gdp_10-14?resource=download

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect

# Extract

## Store CSV into DataFrame

In [ ]:
#Load in energy data
csv_file_1 = "resources/energy_census_and_economic_data_US_2010-2014.csv"
energy_df = pd.read_csv(csv_file_1)
energy_df.head()

In [ ]:
# energy_columns = energy_df.columns

# for column in energy_columns:
#     print(column)

In [ ]:
# # Calculate yearly GDP rates and adding the columns into the energy_df dataframe.
# # yearly GDP change. Calculated by dividing the second year’s GDP by the first year’s GDP.
# gdp_2010_2011 = []
# gdp_2011_2012 = []
# gdp_2012_2013 = []
# gdp_2013_2014 = []

# i = 0
# while i < len(energy_df):
#     if True:
#         gdp_2010_2011.append(energy_df["GDP2011"][i] / energy_df["GDP2010"][i] * 100)
#         gdp_2011_2012.append(energy_df["GDP2012"][i] / energy_df["GDP2011"][i] * 100)
#         gdp_2012_2013.append(energy_df["GDP2013"][i] / energy_df["GDP2012"][i] * 100)
#         gdp_2013_2014.append(energy_df["GDP2014"][i] / energy_df["GDP2013"][i] * 100)
#     i += 1
    
# energy_df["GDP_2010_2011"] = gdp_2010_2011
# energy_df["GDP_2011_2012"] = gdp_2011_2012
# energy_df["GDP_2012_2013"] = gdp_2012_2013
# energy_df["GDP_2013_2014"] = gdp_2013_2014
    
# gdp_df = energy_df[['StateCodes', 'State', 'GDP_2010_2011', 'GDP_2011_2012', 'GDP_2012_2013', 'GDP_2013_2014' ]]
# gdp_df = gdp_df.rename(columns={'StateCodes':'statecodes', 'State':'state', 'GDP_2010_2011': 'gdp_2010_2011', 'GDP_2011_2012': 'gdp_2011_2012', 'GDP_2012_2013': 'gdp_2012_2013', 'GDP_2013_2014': 'gdp_2013_2014' }).copy()
# gdp_df

# Transform

In [ ]:
#Cleaning Energy Consumption Data
year = []
state = []
#Total energy consumption:
energy_consumption = []
#Sectors of energy consumption:
biomass_consumption = []
coal_consumption = []
elec_consumption = []
fossfuel_consumption = []
natgas_consumption = []

#GDP by year
gdp = []

year_starter = 2010
year_counter = 0


while True:
    for index, row in energy_df.iterrows():
        if (index == (len(energy_df)-1)):
            year.append(year_starter + year_counter)
            state.append(energy_df['State'][index])
            energy_consumption.append(energy_df[f"TotalC{year_starter + year_counter}"][index])
            biomass_consumption.append(energy_df[f"BiomassC{year_starter + year_counter}"][index])
            print(f"BiomassC{year_starter + year_counter}")
            coal_consumption.append(energy_df[f"CoalC{year_starter + year_counter}"][index])
            elec_consumption.append(energy_df[f"ElecC{year_starter + year_counter}"][index])
            fossfuel_consumption.append(energy_df[f"FossFuelC{year_starter + year_counter}"][index])
            natgas_consumption.append(energy_df[f"NatGasC{year_starter + year_counter}"][index])
            gdp.append(energy_df[f"GDP{year_starter + year_counter}"][index])
            year_counter += 1
            continue

        else:
            year.append(year_starter + year_counter)
            state.append(energy_df['State'][index])
            energy_consumption.append(energy_df[f"TotalC{year_starter + year_counter}"][index])
            biomass_consumption.append(energy_df[f"BiomassC{year_starter + year_counter}"][index])
            coal_consumption.append(energy_df[f"CoalC{year_starter + year_counter}"][index])
            elec_consumption.append(energy_df[f"ElecC{year_starter + year_counter}"][index])
            fossfuel_consumption.append(energy_df[f"FossFuelC{year_starter + year_counter}"][index])
            natgas_consumption.append(energy_df[f"NatGasC{year_starter + year_counter}"][index])
            gdp.append(energy_df[f"GDP{year_starter + year_counter}"][index])
    
    if (year_starter + year_counter == 2015):
        break

In [ ]:
energy_cleaned_df = pd.DataFrame({
#     'year_state': year_state,
    'year': year,
    'state': state,
    'total_energy': energy_consumption,
    'gdp': gdp,
    'biomass_consumption': biomass_consumption,
    'coal_consumption': coal_consumption,
    'electricity_consumption': elec_consumption,
    'fossil_fuel_consumption': fossfuel_consumption,
    'natural_gas_consumption': natgas_consumption
})

energy_cleaned_df

In [ ]:
energy_cleaned_df.columns

## Create database connection

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = 'bootcamp'
host = 'localhost'
port = 5432
database_name = 'energy_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
# Confirm tables
inspector = inspect(engine)
inspector.get_table_names()

## Load DataFrames into database

In [ ]:
energy_cleaned_df.to_sql(name='energy_consumption', con=engine, if_exists='append', index=False)

In [ ]:
# cleaned_min_wage_data_df.to_sql(name='minimum_wage', con=engine, if_exists='append', index=False)

## Confirm data has been added by querying the DB table

In [ ]:
pd.read_sql_query('select * from energy_consumption', con=engine).head()

In [ ]:
# Export the DataFrames to CSV

energy_cleaned_df.to_csv("energy_cleaned_df.csv", index=True)